# Experiment 001: Baseline from Pre-computed Solution

Goal: Evaluate the pre-computed solution from SmartManoj's GitHub and compare with sample_submission.csv

In [ ]:
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
from shapely.strtree import STRtree
from shapely import affinity
from decimal import Decimal, getcontext
import warnings
warnings.filterwarnings('ignore')

getcontext().prec = 30

print("Libraries loaded successfully")

In [ ]:
class ChristmasTree:
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        
        # Tree dimensions
        trunk_w, trunk_h = Decimal("0.15"), Decimal("0.2")
        base_w, mid_w, top_w = Decimal("0.7"), Decimal("0.4"), Decimal("0.25")
        tip_y, tier_1_y, tier_2_y = Decimal("0.8"), Decimal("0.5"), Decimal("0.25")
        base_y, trunk_bottom_y = Decimal("0.0"), -trunk_h
        
        # 15 vertices
        initial_polygon = Polygon([
            (0, float(tip_y)),
            (float(top_w/2), float(tier_1_y)),
            (float(top_w/4), float(tier_1_y)),
            (float(mid_w/2), float(tier_2_y)),
            (float(mid_w/4), float(tier_2_y)),
            (float(base_w/2), float(base_y)),
            (float(trunk_w/2), float(base_y)),
            (float(trunk_w/2), float(trunk_bottom_y)),
            (float(-trunk_w/2), float(trunk_bottom_y)),
            (float(-trunk_w/2), float(base_y)),
            (float(-base_w/2), float(base_y)),
            (float(-mid_w/4), float(tier_2_y)),
            (float(-mid_w/2), float(tier_2_y)),
            (float(-top_w/4), float(tier_1_y)),
            (float(-top_w/2), float(tier_1_y)),
        ])
        
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x), yoff=float(self.center_y))

def has_overlap(trees):
    """Check if any trees overlap (touching is OK)"""
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def get_score(trees, n):
    """Calculate score for a single n-tree configuration"""
    xys = np.concatenate([np.asarray(t.polygon.exterior.xy).T for t in trees])
    side = max(xys.max(axis=0) - xys.min(axis=0))
    return side**2 / n

print("ChristmasTree class and helper functions defined")

In [ ]:
def parse_submission(df):
    """Parse submission dataframe and return trees grouped by n"""
    groups = {}
    for n in range(1, 201):
        group = df[df["id"].str.startswith(f"{n:03d}_")]
        if len(group) > 0:
            trees = []
            for _, row in group.iterrows():
                x = str(row["x"])[1:]  # Remove 's' prefix
                y = str(row["y"])[1:]
                deg = str(row["deg"])[1:]
                trees.append(ChristmasTree(x, y, deg))
            groups[n] = trees
    return groups

def calculate_total_score(groups, verbose=False):
    """Calculate total score and check for overlaps"""
    total_score = 0.0
    overlaps = []
    scores_by_n = {}
    
    for n in range(1, 201):
        if n in groups:
            trees = groups[n]
            score = get_score(trees, n)
            scores_by_n[n] = score
            total_score += score
            
            if has_overlap(trees):
                overlaps.append(n)
                if verbose:
                    print(f"  n={n}: OVERLAP DETECTED!")
            elif verbose and n <= 10:
                print(f"  n={n}: score={score:.6f}")
    
    return total_score, overlaps, scores_by_n

print("Scoring functions defined")

In [ ]:
# Load pre-computed solution from GitHub
print("Loading pre-computed solution from GitHub...")
precomputed_df = pd.read_csv('/home/code/precomputed_submission.csv')
print(f"Shape: {precomputed_df.shape}")
print(precomputed_df.head())

In [ ]:
# Parse and score pre-computed solution
print("\nParsing pre-computed solution...")
precomputed_groups = parse_submission(precomputed_df)
print(f"Parsed {len(precomputed_groups)} groups")

print("\nCalculating score for pre-computed solution...")
precomputed_score, precomputed_overlaps, precomputed_scores = calculate_total_score(precomputed_groups, verbose=True)
print(f"\nPre-computed solution total score: {precomputed_score:.6f}")
print(f"Overlapping configurations: {len(precomputed_overlaps)}")
if precomputed_overlaps:
    print(f"Overlap n values: {precomputed_overlaps[:20]}..." if len(precomputed_overlaps) > 20 else f"Overlap n values: {precomputed_overlaps}")

In [ ]:
# Load sample submission
print("Loading sample submission...")
sample_df = pd.read_csv('/home/data/sample_submission.csv')
print(f"Shape: {sample_df.shape}")

print("\nParsing sample submission...")
sample_groups = parse_submission(sample_df)
print(f"Parsed {len(sample_groups)} groups")

print("\nCalculating score for sample submission...")
sample_score, sample_overlaps, sample_scores = calculate_total_score(sample_groups, verbose=True)
print(f"\nSample submission total score: {sample_score:.6f}")
print(f"Overlapping configurations: {len(sample_overlaps)}")

In [ ]:
# Compare scores
print("\n" + "="*60)
print("SCORE COMPARISON")
print("="*60)
print(f"Pre-computed solution (GitHub): {precomputed_score:.6f}")
print(f"Sample submission:               {sample_score:.6f}")
print(f"Target to beat:                  68.931058")
print(f"\nImprovement over sample: {sample_score - precomputed_score:.6f}")
print(f"Gap to target: {precomputed_score - 68.931058:.6f}")

In [ ]:
# Analyze score distribution by n
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Score by n
ns = list(range(1, 201))
precomputed_scores_list = [precomputed_scores.get(n, 0) for n in ns]
sample_scores_list = [sample_scores.get(n, 0) for n in ns]

axes[0].plot(ns, precomputed_scores_list, label='Pre-computed', alpha=0.7)
axes[0].plot(ns, sample_scores_list, label='Sample', alpha=0.7)
axes[0].set_xlabel('n (number of trees)')
axes[0].set_ylabel('Score contribution (s²/n)')
axes[0].set_title('Score by Configuration Size')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Cumulative score
cum_precomputed = np.cumsum(precomputed_scores_list)
cum_sample = np.cumsum(sample_scores_list)

axes[1].plot(ns, cum_precomputed, label='Pre-computed', alpha=0.7)
axes[1].plot(ns, cum_sample, label='Sample', alpha=0.7)
axes[1].axhline(y=68.931058, color='r', linestyle='--', label='Target')
axes[1].set_xlabel('n (number of trees)')
axes[1].set_ylabel('Cumulative Score')
axes[1].set_title('Cumulative Score')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/home/code/experiments/001_baseline/score_comparison.png', dpi=100)
plt.show()
print("Score comparison plot saved")

In [ ]:
# If pre-computed solution is valid (no overlaps), use it as submission
if len(precomputed_overlaps) == 0:
    print("Pre-computed solution has no overlaps - using as submission")
    precomputed_df.to_csv('/home/submission/submission.csv', index=False)
    print("Saved to /home/submission/submission.csv")
else:
    print(f"Pre-computed solution has {len(precomputed_overlaps)} overlapping configurations")
    print("Need to repair overlaps before submission")

In [ ]:
# Verify submission file
print("\nVerifying submission file...")
submission = pd.read_csv('/home/submission/submission.csv')
print(f"Shape: {submission.shape}")
print(f"Columns: {submission.columns.tolist()}")
print(f"\nFirst 5 rows:")
print(submission.head())
print(f"\nLast 5 rows:")
print(submission.tail())

In [ ]:
# Final summary
print("\n" + "="*60)
print("EXPERIMENT 001 SUMMARY")
print("="*60)
print(f"Pre-computed solution score: {precomputed_score:.6f}")
print(f"Target score: 68.931058")
print(f"Gap to target: {precomputed_score - 68.931058:.6f}")
print(f"Overlaps: {len(precomputed_overlaps)}")
print(f"Submission saved: /home/submission/submission.csv")